In [ ]:
%load_ext sql

# Connect to CHINOOK database

Conect directly from sqlite folder

In [ ]:
%sql sqlite:///C:/sqlite/chinook.db

In [ ]:
%%sql
PRAGMA table_info(track);

 * sqlite:///C:/sqlite/chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,track_id,INTEGER,1,None,1
1,name,NVARCHAR(200),1,None,0
2,album_id,INTEGER,0,None,0
3,media_type_id,INTEGER,1,None,0
4,genre_id,INTEGER,0,None,0
5,composer,NVARCHAR(220),0,None,0
6,milliseconds,INTEGER,1,None,0
7,bytes,INTEGER,0,None,0
8,unit_price,"NUMERIC(10,2)",1,None,0
